# Regrouper les données

L'analyse de données s'intéresse souvent à grouper des données suivant un critère, le salaire suivant l'âge, le métier suivant le genre, les dividendes des entreprises suivant le pays etc.

Pour faire cela Pandas offre la méthode [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html). Cette méthode

* découpe les données en groupes basés sur un critère
* applique une fonction sur chaque groupe indépendamment
* combine les résultats en un DataFrame

In [1]:
import pandas as pd
import numpy as np
np.random.seed(2)
pd.set_option('precision', 3)

size = 20
df = pd.DataFrame({'A': np.random.randn(size), 
                   'B': np.random.randint(5,size=size),
                   'C': np.random.randint(5,size=size)})
df.B += 3
df.head()

,A,B,C
0,-0.417,6,1
1,-0.056,4,4
2,-2.136,5,2
3,1.640,3,3
4,-1.793,7,0


Les valeurs de B étant limitées, regroupons nos données pour chaque valeur de B. 

Une fois ce choix décidé il est nécessaire de savoir ce qu'on fait dans les autres colonnes lorsque plusieurs lignes ont la même valeur B. Ici on choisit
de calculer la moyenne pour toutes les autres colonnes. Ensuite on ajoute une nouvelle colonne qui calcule la taille
de chaque groupe.

In [2]:
df2 = df.groupby('B').mean()
df2['countB'] = df.groupby('B').size()
df2

,A,C,countB
B,,,
3,0.678,3.500,4
4,-0.570,1.667,6
5,0.136,1.000,6
6,-0.417,1.000,1
7,-1.293,1.667,3


#### Structure

Le contenu de chaque groupe est stocké dans `groups`. Pour voir les données d'un groupe dans un tableau on utilisera `get_group`.

In [3]:
df.groupby('B').groups

{3: Int64Index([3, 10, 14, 16], dtype='int64'),
 4: Int64Index([1, 9, 13, 15, 18, 19], dtype='int64'),
 5: Int64Index([2, 6, 8, 11, 12, 17], dtype='int64'),
 6: Int64Index([0], dtype='int64'),
 7: Int64Index([4, 5, 7], dtype='int64')}

In [4]:
df.groupby('B').get_group(7)

,A,B,C
4,-1.793,7,0
5,-0.842,7,3
7,-1.245,7,2


### Grouper suivant plusieurs colonnes

On peut aussi indiquer plusieurs champs pour regrouper les données ce qui donne des index et sous-index.

In [5]:
df.groupby(['B','C']).first()  # get first value of A for each group

A
B C       
3 3  1.640
  4  0.551
4 0 -0.909
  1 -0.596
  2 -1.118
  4 -0.056
5 0  0.503
  2 -2.136
6 1 -0.417
7 0 -1.793
  2 -1.245
  3 -0.842

In [6]:
df.groupby(['B','C']).get_group((3,3))

,A,B,C
3,1.640,3,3
16,-0.019,3,3


## Grouper suivant un sous-index

Il est aussi possible de grouper suivant les valeurs de l'index ou d'un sous index. Pour cela il faut indiquer
le niveau de l'index à la place du nom de la colonne.

In [7]:
dfm = df.groupby(['B','C']).first()
dfm

A
B C       
3 3  1.640
  4  0.551
4 0 -0.909
  1 -0.596
  2 -1.118
  4 -0.056
5 0  0.503
  2 -2.136
6 1 -0.417
7 0 -1.793
  2 -1.245
  3 -0.842

In [8]:
dfm.groupby(level=1).sum()

,A
C,
0,-2.200
1,-1.013
2,-4.499
3,0.799
4,0.495


## Appliquer différentes opérations

Il est possible d'appliquer différentes opérations (fonctions) d'un coup :

* une liste de fonctions qui sera appliquée à chaque colonne
* un dictionnaire qui indique quelle méthode appliquer à quelle colonne (très utile)

In [9]:
df.groupby('B').agg([np.mean, 'last'])  # some function are predefined and therefore can be named

A             C     
    mean   last   mean last
B                          
3  0.678 -0.019  3.500    3
4 -0.570  0.009  1.667    1
5  0.136  1.175  1.000    0
6 -0.417 -0.417  1.000    1
7 -1.293 -1.245  1.667    2

In [10]:
df.groupby('B').agg({'A': np.sum, 'C': lambda x : x[x%2 == 0].mean() })

,A,C
B,,
3,2.712,4.0
4,-3.418,2.0
5,0.817,1.0
6,-0.417,NaN
7,-3.880,1.0


## Un groupby qui conserve la structure initiale

Un problème avec `groupby` est qu'on pert le nombre de lignes initial ce qui complique les choses
si on désire repporter le résultat dans le tableau initial.

Imaginons que je désire ajouter à mon tableau une colonne qui soit la moyenne mensuelle pour chaque
valeur afin de relativiser mes valeurs (par rapport à la moyenne). 

Avec `groupby` j'ai bien la moyenne mais je n'ai plus mes valeurs initiale. Je pourrais faire une opération
compliquée pour ajouter une colonne au tableau initial dans laquelle je recopie la bonne valeur du `groupby`.
Je peux aussi utiliser `transform` qui fait ce travail.

Remplacer la fonction de réduction `f` sur les groupes par `transform(f)` permet de conserver le nombre de
lignes du tableau donné.

In [11]:
df = pd.DataFrame({'month': np.random.randint(1,4,size=10), 
                   'day sales': np.random.randint(50,size=10)}).sort_values('month')

df

,month,day sales
0,1,20
1,1,26
2,2,23
3,2,22
5,2,37
6,2,10
7,2,8
8,2,26
4,3,43
9,3,35


In [12]:
df.groupby('month').mean()

,day sales
month,
1,23
2,21
3,39


In [13]:
df.groupby('month').transform(np.mean)

,day sales
0,23
1,23
2,21
3,21
5,21
6,21
7,21
8,21
4,39
9,39


In [14]:
df['mean day sales'] = df.groupby('month').transform(np.mean)['day sales']
df

,month,day sales,mean day sales
0,1,20,23
1,1,26,23
2,2,23,21
3,2,22,21
5,2,37,21
6,2,10,21
7,2,8,21
8,2,26,21
4,3,43,39
9,3,35,39


{{ PreviousNext("pd04 -- N-dimensions dataframe or multi-index.ipynb", "pd06 -- Merging 2 dataframes.ipynb")}}